<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB31_Planning_vs_Reactive_Agents_avec_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB31 : Planning vs Reactive Loops avec LangChain

**Objectif:** Construire deux agents LangChain simples:
- **Reactive Agent** → répond directement, outil par outil
- **Planning Agent** → décompose en sous-tâches, puis exécute

**Durée estimée:** 20-30 minutes

**Livrable:** Notebook comparant les sorties des deux patterns sur les mêmes requêtes

## Step 1: Setup

Installation des dépendances nécessaires

In [1]:
# Installation des packages
!pip install -q langchain langchain-openai openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.5 MB/s eta 0:00:00


In [2]:
# Configuration de la clé API OpenAI
# Dans Google Colab, nous utilisons les secrets au lieu de .env
import os
from google.colab import userdata

# Option 1: Utiliser les secrets Colab (recommandé)
# Allez dans le menu à gauche (icône clé) > Secrets > Ajouter "OPENAI_API_KEY"
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ Clé API chargée depuis les secrets Colab")
except:
    print("⚠️ Secret non trouvé. Utilisation de l'input manuel...")
    # Option 2: Input manuel (moins sécurisé)
    import getpass
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Entrez votre clé API OpenAI: ')

✅ Clé API chargée depuis les secrets Colab


## Step 2: Définition des Tools

Création de deux outils simples:
1. **Calculator** - Effectue des calculs mathématiques
2. **Knowledge Base Lookup** - Recherche dans une base de connaissances

In [17]:
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# Outil 1: Calculatrice
def calculator(expr: str) -> str:
    """
    Évalue une expression mathématique.
    Exemple: "15*4" -> "60"
    """
    try:
        result = eval(expr)
        return str(result)
    except Exception as e:
        return f"Erreur: {e}"

# Outil 2: Base de connaissances
kb = {
    "LangChain": "Un framework pour construire des agents alimentés par des LLM.",
    "RAG": "Retrieval-Augmented Generation, combinant récupération et génération.",
    "CrewAI": "Un framework pour l'orchestration multi-agents."
}

def lookup(term: str) -> str:
    """
    Recherche un terme dans la base de connaissances.
    """
    return kb.get(term, f"Aucune entrée pour '{term}'")

# Définition des outils pour LangChain
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'."
    ),
]

print("✅ Outils définis:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

✅ Outils définis:
  - Calculator: Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'.
  - Lookup: Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'.


### Test des outils

In [16]:
# Test de la calculatrice
print("Test Calculator:")
print(f"  15*4 = {calculator('15*4')}")
print(f"  25*3+10 = {calculator('25*3+10')}")
print(f"  12/4 = {calculator('12/4')}")

print("\nTest Lookup:")
print(f"  LangChain: {lookup('LangChain')}")
print(f"  RAG: {lookup('RAG')}")
print(f"  CrewAI: {lookup('CrewAI')}")
print(f"  Unknown: {lookup('Unknown')}")

Test Calculator:
  15*4 = 60
  25*3+10 = 85
  12/4 = 3.0

Test Lookup:
  LangChain: Un framework pour construire des agents alimentés par des LLM.
  RAG: Retrieval-Augmented Generation, combinant récupération et génération.
  CrewAI: Un framework pour l'orchestration multi-agents.
  Unknown: Aucune entrée pour 'Unknown'


## Step 3: Reactive Agent (20 min)

Un agent réactif simple: le LLM décide quel outil utiliser → exécute → répond.

Le pattern **ReAct** (Reasoning + Acting) est utilisé:
1. **Thought**: L'agent réfléchit à la prochaine action
2. **Action**: Il exécute un outil
3. **Observation**: Il observe le résultat
4. Répète jusqu'à avoir la réponse finale

In [19]:
!pip install langgraph -q


In [58]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Define your tools (example)
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """Useful for mathematical calculations. Input should be a valid Python expression like '15*4'"""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

tools = [calculator]

# Create the agent (LangGraph's create_react_agent doesn't need a prompt template)
agent_executor = create_react_agent(llm, tools)

# Run the agent
print("\n--- Reactive Agent ---")
result = agent_executor.invoke({
    "messages": [("user", "What is 15*4 and also explain what LangChain is?")]
})

# Print the final response
print(result["messages"][-1].content)

/tmp/ipython-input-4090695859.py:22: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)



--- Reactive Agent ---
The result of \( 15 \times 4 \) is \( 60 \).

### What is LangChain?

LangChain is a framework designed for developing applications that utilize large language models (LLMs). It provides a structured way to build applications that can interact with LLMs, enabling developers to create more complex and capable systems. Key features of LangChain include:

1. **Modularity**: LangChain allows developers to compose different components, such as prompt templates, memory, and chains, to create sophisticated workflows.

2. **Integration**: It supports integration with various data sources and APIs, making it easier to pull in external information and enhance the capabilities of LLMs.

3. **Chains**: LangChain enables the creation of chains, which are sequences of calls to LLMs or other functions, allowing for more complex interactions and processing.

4. **Memory**: It provides mechanisms for memory, allowing applications to remember past interactions and maintain contex

In [64]:
from langchain.agents import create_agent
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialisation du LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Définition du prompt React
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Création de l'agent réactif
agent = create_react_agent(llm, tools)


print("✅ Agent réactif créé avec succès")

✅ Agent réactif créé avec succès


/tmp/ipython-input-2921791263.py:33: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [65]:
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


# Vos outils (inchangés)
def calculator(expr: str) -> str:
    """Évalue une expression mathématique."""
    try:
        result = eval(expr)
        return str(result)
    except Exception as e:
        return f"Erreur: {e}"

kb = {
    "LangChain": "Un framework pour construire des agents alimentés par des LLM.",
    "RAG": "Retrieval-Augmented Generation, combinant récupération et génération.",
    "CrewAI": "Un framework pour l'orchestration multi-agents."
}

def lookup(term: str) -> str:
    """Recherche un terme dans la base de connaissances."""
    return kb.get(term, f"Aucune entrée pour '{term}'")

tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Utilisé pour effectuer des calculs mathématiques. Input: une expression mathématique comme '15*4' ou '25*3+10'."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Utilisé pour rechercher des informations dans la base de connaissances. Input: un terme comme 'LangChain', 'RAG', ou 'CrewAI'."
    ),
]

# Initialisation du LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Création de l'agent avec LangGraph
reactive_agent = create_react_agent(llm, tools)

print("✅ Agent réactif créé avec succès")

# Utilisation de l'agent
response = reactive_agent.invoke({
    "messages": [("user", "Que représente LangChain et calcule 15*4")]
})

print("\n📝 Réponse:", response["messages"][-1].content)


/tmp/ipython-input-3819649997.py:42: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  reactive_agent = create_react_agent(llm, tools)


✅ Agent réactif créé avec succès

📝 Réponse: LangChain est un framework pour construire des agents alimentés par des modèles de langage (LLM). 

En ce qui concerne le calcul, \( 15 \times 4 = 60 \).


### Test du Reactive Agent

In [66]:
print("\n" + "="*80)
print("REACTIVE AGENT - Query 1")
print("="*80)

query1 = "Qu'est-ce que 15*4 et explique aussi ce qu'est LangChain?"
# result1 = reactive_agent.invoke(query1)
result1 = reactive_agent.invoke({
    "messages": [("user", query1)]
})
print("\n" + "-"*80)
print("RÉSULTAT FINAL:")
print("-"*80)
print("\n📝 Réponse:\n\n", result1["messages"][-1].content)




REACTIVE AGENT - Query 1

--------------------------------------------------------------------------------
RÉSULTAT FINAL:
--------------------------------------------------------------------------------

📝 Réponse:

 Le résultat de \( 15 \times 4 \) est \( 60 \).

LangChain est un framework conçu pour construire des agents alimentés par des modèles de langage (LLM).


**👉 Observation:** L'agent réactif interagit pas à pas:
- Il analyse la question
- Décide d'utiliser un outil
- Observe le résultat
- Décide de la prochaine action
- Continue jusqu'à avoir une réponse complète

## Step 4: Planning Agent (25 min)

Un agent planificateur décompose d'abord la requête en sous-tâches, puis les exécute.

**Planning = décomposer la requête en sous-tâches d'abord, puis exécuter**

In [71]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Template de prompt pour la planification
plan_prompt = PromptTemplate.from_template("""
Tu es un agent de planification. Décompose la requête suivante en sous-tâches avant de répondre:
Requête: {task}
Retourne le plan sous la forme:
1. Sous-tâche A
2. Sous-tâche B
...
Sois précis et indique clairement les outils nécessaires (Calculator ou Lookup).
""")

# Chaîne de planification (syntaxe LCEL)
plan_chain = plan_prompt | llm | StrOutputParser()

def planning_agent(task: str) -> str:
    """
    Agent planificateur: crée un plan puis l'exécute.
    """
    # Étape 1: Créer le plan
    print("\n🧠 PHASE DE PLANIFICATION...")
    plan = plan_chain.invoke({"task": task})
    print("\n📋 PLAN GÉNÉRÉ:")
    print("-" * 60)
    print(plan)
    print("-" * 60)

    # Étape 2: Exécution simple (détection par mots-clés)
    print("\n⚙️ PHASE D'EXÉCUTION...")
    answer_parts = []

    # Détection de calculs
    import re
    calc_patterns = re.findall(r'\d+[\*\/\+\-]\d+', task)
    if calc_patterns or "calcul" in task.lower() or "*" in task:
        # Extraire l'expression mathématique
        for expr in calc_patterns:
            result = calculator(expr)
            answer_parts.append(f"📊 Calcul de {expr}: {result}")

    # Détection de termes à rechercher
    for term in kb.keys():
        if term in task:
            result = lookup(term)
            answer_parts.append(f"📚 Information sur {term}: {result}")

    print("\n✅ RÉSULTATS D'EXÉCUTION:")
    for part in answer_parts:
        print(f"  {part}")

    return "\n".join(answer_parts)
    print("✅ Agent planificateur créé avec succès")

### Test du Planning Agent

In [72]:
print("\n" + "="*80)
print("PLANNING AGENT - Query 1")
print("="*80)

query1 = "Qu'est-ce que 15*4 et explique aussi ce qu'est LangChain?"
result1 = planning_agent(query1)

print("\n" + "="*80)
print("RÉPONSE FINALE:")
print("="*80)
print(result1)


PLANNING AGENT - Query 1

🧠 PHASE DE PLANIFICATION...

📋 PLAN GÉNÉRÉ:
------------------------------------------------------------
Plan de décomposition de la requête :

1. Sous-tâche A : Calculer le produit de 15 et 4.
   - Outil nécessaire : Calculator

2. Sous-tâche B : Rechercher des informations sur LangChain.
   - Outil nécessaire : Lookup

Une fois ces sous-tâches réalisées, je pourrai fournir la réponse complète à la requête.
------------------------------------------------------------

⚙️ PHASE D'EXÉCUTION...

✅ RÉSULTATS D'EXÉCUTION:
  📊 Calcul de 15*4: 60
  📚 Information sur LangChain: Un framework pour construire des agents alimentés par des LLM.

RÉPONSE FINALE:
📊 Calcul de 15*4: 60
📚 Information sur LangChain: Un framework pour construire des agents alimentés par des LLM.


**👉 Observation:** L'agent planificateur:
1. **D'abord** crée un plan structuré
2. **Ensuite** exécute les sous-tâches en séquence
3. Structure globale plus claire

## Step 5: Compare Outputs (15 min)

Comparaison directe des deux agents sur les mêmes requêtes

In [75]:
# Requêtes de test
test_queries = [
    "Qu'est-ce que 25*3+10?",
    "Dis-moi ce qu'est CrewAI, puis calcule 12/4.",
    "Calcule 100*2 et explique RAG."
]

print("\n" + "#"*80)
print("# COMPARAISON DES DEUX AGENTS")
print("#"*80)

for i, query in enumerate(test_queries, 1):
    print(f"\n\n{'='*80}")
    print(f"QUERY {i}: {query}")
    print("="*80)

    # Reactive Agent
    print("\n" + "▶"*40)
    print("REACTIVE AGENT")
    print("▶"*40)
    try:
        reactive_result = reactive_agent.invoke(query)
        print(f"\n✅ Résultat: {reactive_result}")
    except Exception as e:
        print(f"\n❌ Erreur: {e}")

    # Planning Agent
    print("\n" + "▶"*40)
    print("PLANNING AGENT")
    print("▶"*40)
    try:
        planning_result = planning_agent(query)
        print(f"\n✅ Résultat: {planning_result}")
    except Exception as e:
        print(f"\n❌ Erreur: {e}")


################################################################################
# COMPARAISON DES DEUX AGENTS
################################################################################


QUERY 1: Qu'est-ce que 25*3+10?

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
REACTIVE AGENT
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

❌ Erreur: Expected dict, got Qu'est-ce que 25*3+10?
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_GRAPH_NODE_RETURN_VALUE

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
PLANNING AGENT
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

🧠 PHASE DE PLANIFICATION...

📋 PLAN GÉNÉRÉ:
------------------------------------------------------------
Pour répondre à la requête "Qu'est-ce que 25*3+10?", voici le plan décomposé en sous-tâches :

1. Sous-tâche A : Calculer le produit de 25 et 3.
   - Outil nécessaire : Calculator

2. Sous-tâche B : Ajouter 10 au résultat obtenu de la sous-tâche A.
   - Outil nécessaire : Calculator

3. Sous-tâche C : 

### Observations sur les différences

**Reactive Agent:**
- ✅ Répond rapidement et directement
- ✅ Adapte dynamiquement sa stratégie
- ⚠️ Peut perdre la structure globale
- ⚠️ Raisonnement moins visible

**Planning Agent:**
- ✅ Structure claire et explicite
- ✅ Décomposition visible des tâches
- ✅ Meilleure fiabilité sur tâches complexes
- ⚠️ Prend plus de temps (2 appels LLM)
- ⚠️ Moins flexible si le plan initial est incorrect

## Step 6: Mini-projet - Enhanced Planning Agent :

Amélioration de l'agent planificateur avec:
1. Parsing des sous-tâches avec regex
2. Ajout d'outils supplémentaires
3. Exécution séquentielle avec résultats inter-étapes

### Ajout de nouveaux outils

In [76]:
# Nouvel outil: Résumeur de texte
def summarizer(text: str) -> str:
    """
    Résume un texte (simulation simple).
    """
    words = text.split()
    if len(words) <= 10:
        return text
    return " ".join(words[:10]) + "..."

# Nouvel outil: Recherche web (simulée)
def web_search(query: str) -> str:
    """
    Simule une recherche web.
    """
    mock_results = {
        "python": "Python est un langage de programmation interprété, de haut niveau.",
        "ai": "L'Intelligence Artificielle (IA) est la simulation de processus d'intelligence humaine.",
        "langchain": "LangChain est un framework pour développer des applications alimentées par des LLM."
    }

    for key, value in mock_results.items():
        if key in query.lower():
            return value
    return f"Aucun résultat trouvé pour '{query}'"

# Outils enrichis
enhanced_tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Calcule des expressions mathématiques."
    ),
    Tool(
        name="Lookup",
        func=lookup,
        description="Recherche dans la base de connaissances."
    ),
    Tool(
        name="Summarizer",
        func=summarizer,
        description="Résume un texte long."
    ),
    Tool(
        name="WebSearch",
        func=web_search,
        description="Recherche des informations sur le web (simulé)."
    ),
]

print("✅ Nouveaux outils ajoutés:")
for tool in enhanced_tools:
    print(f"  - {tool.name}")

✅ Nouveaux outils ajoutés:
  - Calculator
  - Lookup
  - Summarizer
  - WebSearch


### Enhanced Planning Agent avec parsing avancé

In [77]:
import re
from typing import List, Dict

# Prompt amélioré
enhanced_plan_prompt = PromptTemplate.from_template("""
Tu es un agent de planification avancé. Décompose la requête suivante en sous-tâches numérotées.

Outils disponibles:
- Calculator: pour les calculs mathématiques
- Lookup: pour la base de connaissances (LangChain, RAG, CrewAI)
- Summarizer: pour résumer du texte
- WebSearch: pour rechercher des informations

Requête: {task}

Format du plan:
1. [NOM_OUTIL] Description de la sous-tâche
2. [NOM_OUTIL] Description de la sous-tâche
...

Exemple:
1. [Calculator] Calculer 15*4
2. [Lookup] Rechercher LangChain
""")

enhanced_plan_chain = LLMChain(llm=llm, prompt=enhanced_plan_prompt)

def parse_plan(plan_text: str) -> List[Dict[str, str]]:
    """
    Parse le plan en sous-tâches structurées.
    Retourne une liste de dicts: [{"tool": "Calculator", "task": "calculer 15*4"}, ...]
    """
    subtasks = []
    # Pattern: "1. [ToolName] description"
    pattern = r'\d+\.\s*\[([A-Za-z]+)\]\s*(.+)'

    for line in plan_text.split('\n'):
        match = re.search(pattern, line)
        if match:
            tool_name = match.group(1)
            task_desc = match.group(2).strip()
            subtasks.append({"tool": tool_name, "task": task_desc})

    return subtasks

def execute_subtask(tool_name: str, task_desc: str) -> str:
    """
    Exécute une sous-tâche avec l'outil approprié.
    """
    # Mapping des outils
    tool_map = {
        "Calculator": calculator,
        "Lookup": lookup,
        "Summarizer": summarizer,
        "WebSearch": web_search
    }

    tool_func = tool_map.get(tool_name)
    if not tool_func:
        return f"❌ Outil '{tool_name}' non trouvé"

    # Extraction de l'input pour l'outil
    if tool_name == "Calculator":
        # Extraire l'expression mathématique
        expr_match = re.search(r'([\d+\-*/\(\)\s]+)', task_desc)
        if expr_match:
            return tool_func(expr_match.group(1).strip())
    elif tool_name == "Lookup":
        # Extraire le terme
        for term in kb.keys():
            if term.lower() in task_desc.lower():
                return tool_func(term)
    elif tool_name == "WebSearch":
        # Utiliser toute la description comme requête
        return tool_func(task_desc)

    return tool_func(task_desc)

def enhanced_planning_agent(task: str) -> str:
    """
    Agent planificateur amélioré avec parsing et exécution séquentielle.
    """
    print("\n" + "="*80)
    print("🚀 ENHANCED PLANNING AGENT")
    print("="*80)

    # Phase 1: Planification
    print("\n📝 Phase 1: PLANIFICATION")
    print("-"*80)
    plan_text = enhanced_plan_chain.run(task)
    print(plan_text)

    # Phase 2: Parsing
    print("\n🔍 Phase 2: PARSING DU PLAN")
    print("-"*80)
    subtasks = parse_plan(plan_text)
    print(f"Nombre de sous-tâches identifiées: {len(subtasks)}")
    for i, st in enumerate(subtasks, 1):
        print(f"  {i}. [{st['tool']}] {st['task']}")

    # Phase 3: Exécution
    print("\n⚙️ Phase 3: EXÉCUTION SÉQUENTIELLE")
    print("-"*80)
    results = []
    for i, subtask in enumerate(subtasks, 1):
        print(f"\n  Étape {i}/{len(subtasks)}: [{subtask['tool']}] {subtask['task']}")
        result = execute_subtask(subtask['tool'], subtask['task'])
        print(f"  ➜ Résultat: {result}")
        results.append(f"Étape {i}: {result}")

    # Phase 4: Synthèse
    print("\n" + "="*80)
    print("✅ RÉSULTATS FINAUX")
    print("="*80)
    final_answer = "\n".join(results)
    print(final_answer)

    return final_answer

print("✅ Enhanced Planning Agent créé")

NameError: name 'LLMChain' is not defined

### Test de l'Enhanced Planning Agent

In [78]:
# Test avec des requêtes complexes
complex_queries = [
    "Calcule 50*2, puis recherche des infos sur Python, et résume le résultat",
    "Qu'est-ce que LangChain? Calcule aussi 144/12 et recherche AI sur le web"
]

for i, query in enumerate(complex_queries, 1):
    print(f"\n\n{'#'*80}")
    print(f"TEST {i}")
    print(f"Query: {query}")
    print("#"*80)

    result = enhanced_planning_agent(query)
    print("\n" + "="*80 + "\n")



################################################################################
TEST 1
Query: Calcule 50*2, puis recherche des infos sur Python, et résume le résultat
################################################################################


NameError: name 'enhanced_planning_agent' is not defined

## Comparaison finale: Les 3 approches

In [ ]:
# Créer un agent réactif avec les outils enrichis pour comparaison
enhanced_reactive_agent = initialize_agent(
    tools=enhanced_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=7,
    handle_parsing_errors=True
)

comparison_query = "Recherche Python sur le web, puis calcule 25*4"

print("\n" + "#"*80)
print("# COMPARAISON FINALE DES 3 APPROCHES")
print("#"*80)
print(f"\nQuery: {comparison_query}\n")

# 1. Reactive Agent
print("\n" + "="*80)
print("1️⃣ REACTIVE AGENT (Enhanced)")
print("="*80)
try:
    reactive_result = enhanced_reactive_agent.run(comparison_query)
    print(f"\n✅ Résultat: {reactive_result}")
except Exception as e:
    print(f"\n❌ Erreur: {e}")

# 2. Planning Agent (simple)
print("\n" + "="*80)
print("2️⃣ PLANNING AGENT (Simple)")
print("="*80)
try:
    planning_result = planning_agent(comparison_query)
    print(f"\n✅ Résultat: {planning_result}")
except Exception as e:
    print(f"\n❌ Erreur: {e}")

# 3. Enhanced Planning Agent
print("\n" + "="*80)
print("3️⃣ ENHANCED PLANNING AGENT")
print("="*80)
try:
    enhanced_result = enhanced_planning_agent(comparison_query)
except Exception as e:
    print(f"\n❌ Erreur: {e}")

## 📊 Tableau récapitulatif

| Aspect | Reactive Agent | Planning Agent (Simple) | Enhanced Planning Agent |
|--------|---------------|------------------------|------------------------|
| **Vitesse** | ⚡⚡⚡ Rapide | ⚡⚡ Moyen | ⚡ Plus lent (3 phases) |
| **Structure** | ❌ Implicite | ✅ Explicite | ✅✅ Très explicite |
| **Traçabilité** | ⚠️ Limitée | ✅ Bonne | ✅✅ Excellente |
| **Flexibilité** | ✅✅ Très flexible | ⚠️ Rigide | ✅ Flexible avec structure |
| **Complexité code** | ⚡ Simple (1 ligne) | ⚡⚡ Moyenne | ⚡⚡⚡ Élevée |
| **Fiabilité** | ⚠️ Variable | ✅ Bonne | ✅✅ Très bonne |
| **Debugging** | ❌ Difficile | ✅ Facile | ✅✅ Très facile |
| **Coût (tokens)** | 💰 Moyen | 💰💰 Élevé (2x LLM) | 💰💰 Élevé (2x LLM) |

### 🎯 Quand utiliser chaque approche?

**Reactive Agent:**
- ✅ Requêtes simples et directes
- ✅ Besoin de réponse rapide
- ✅ Contexte changeant dynamiquement
- ❌ Tâches multi-étapes complexes

**Planning Agent:**
- ✅ Tâches complexes nécessitant décomposition
- ✅ Besoin de traçabilité et audit
- ✅ Workflows répétables
- ✅ Debugging et validation importantes
- ❌ Requêtes très simples (overhead inutile)

**Enhanced Planning Agent:**
- ✅ Production avec exigences de qualité
- ✅ Workflows complexes multi-outils
- ✅ Besoin de logs détaillés
- ✅ Orchestration avancée
- ❌ Prototypage rapide

## 🎓 Concepts clés appris

### 1. ReAct Pattern (Reactive)
```
Thought → Action → Observation → Thought → ...
```
- Raisonnement entrelacé avec l'action
- Adaptation dynamique
- Boucle continue jusqu'à résolution

### 2. Plan-and-Execute Pattern
```
Plan → Parse → Execute → Synthesize
```
- Décomposition explicite
- Exécution séquentielle
- Meilleure prédictibilité

### 3. Trade-offs
- **Latence vs Structure**: Plus de structure = plus de temps
- **Flexibilité vs Fiabilité**: Réactivité = flexible mais moins prédictible
- **Coût vs Qualité**: Planning = 2x appels LLM mais meilleure qualité

### 4. Agentic AI Design Patterns
Ces deux patterns sont **complémentaires** dans l'écosystème Agentic AI:
- Reactive pour interactions utilisateur
- Planning pour workflows backend
- Hybride possible: plan global + exécution réactive

## 💡 Exercices supplémentaires

Pour approfondir votre compréhension:

1. **Créer un agent hybride** qui décide dynamiquement s'il doit planifier ou réagir

2. **Ajouter de la mémorisation** entre les étapes du planning agent

3. **Implémenter un système de retry** si une sous-tâche échoue

4. **Créer un orchestrateur** qui distribue les sous-tâches à plusieurs agents spécialisés

5. **Ajouter du logging structuré** pour tracer complètement les décisions

6. **Mesurer les performances** (temps, tokens, précision) des deux approches

7. **Créer un visualisateur** du plan d'exécution (graphe de dépendances)

## ✅ Conclusion

Ce lab vous a permis de:

1. ✅ Comprendre les différences fondamentales entre agents réactifs et planificateurs
2. ✅ Implémenter les deux patterns avec LangChain
3. ✅ Comparer leurs performances sur des tâches réelles
4. ✅ Créer un agent planificateur avancé avec parsing et exécution structurée
5. ✅ Identifier les cas d'usage appropriés pour chaque approche

### Points clés à retenir:

- **Reactive agents** = rapides, flexibles, moins structurés
- **Planning agents** = structurés, traçables, plus fiables sur tâches complexes
- Le choix dépend du **contexte**, des **exigences**, et du **budget**
- Les deux patterns sont **complémentaires** dans une architecture Agentic AI

### Ressources pour aller plus loin:

- [LangChain Documentation - Agents](https://python.langchain.com/docs/modules/agents/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [Plan-and-Solve Paper](https://arxiv.org/abs/2305.04091)
- [LangChain Plan-and-Execute](https://python.langchain.com/docs/use_cases/more/agents/autonomous_agents/plan_and_execute)

---

**🎉 Félicitations! Vous avez complété le Lab 3 sur Planning vs Reactive Loops!**